In [1]:
import pandas as pd

In [2]:
ms = pd.read_csv('../../sample_train.csv', low_memory=False)

## Fazer breve desrição do dataset
## Explicar que foi impossível conseguir carregar o dataset e por isso decidimos sleecionar apenas 600k linhas

## Falar da estratégia que usamos para explorar os dados

### Vimos para cada coluna o número de valores nulos e caso fosse muito alto eliminamos
### Vimos para cada coluna o número de diferentes atributos e caso fosse demasiado alto eliminamos
### Vimos para cada coluna os valores diferentes dos atributos para verificar se era possível serem agrupados
### Vimos para cada coluna que continha NAs a percentagem de cada atributo para verificar se alteravamos pelo valor modal ou era necessário prever
### Vimos os outliers nas numéricoas para verificar se deviamos alterar pela média ou mediana e o mesmo para a standardização

#### Apenas referir detalhadamente os casos relevantes
    - ou seja aqueles em que existiam NAs, ou muitos ateributos e n davam para agrupar, os que davam para agrupar e os q era necessário pever
    - para além disso falar daquele do peixoto q eliminamos por ser parecido a ouro e o do carlos OS_BUILDQQCOISA por ter valores que os outros estavam contidos nele

## Exploração inicial
Inicialmente realizmos uma exploração dos dados para verificar as variáveis do conjunto de dados. Primeiro verificamos quantas linhas completas existiam no *dataset* e concluímos que não existia nenhuma. De seguida, e com o auxílio da descrição das variáveis, verificamos quais variáveis são quantitativas e quais são qualitativas. Chegamos à conclusão que existem apenas 10 variáveis quantitativas, o que mostrou ser um pequeno problema visto que a maior parte dos modelos apenas trabalha com valores numéricos, ou seja, era necessário tratar os dados.

#### Variáveis qualitativas
Para cada coluna qualitativa do dataset verificamos as seguintes informações:
    - número de valores nulos
    - número total de valores diferentes
Após esta análise decidimos eliminar todas as variáveis que continham um número elevado de valores nulos, visto que consideramos que tratar essas variáveis pode ser um erro pelo facto de a informação referente às mesmas ser praticamente nula. Consideramos então que colunas com mais de 95% de NAs seriam intratáveis pelo que optamos por eliminar essas colunas.

De seguida verificamos o número total de valores diferentes para cada coluna. Isto demonstra ser importante visto que caso uma variável categórica apresente muitos valores não agrupáveis, além de não ser muito útil para efetuar previsões, dificulta a plicação de métodos tradicionais de *machine learning* dado que após efetuar *One-Hot-Encoding*, esta variável iria gerar demasiadas colunas. Por esta razão decidimos eliminar todas as colunas com muitos valores distintos não agrupáveis. Verificamos que existiam bastantes colunas com um número elevado de valores, pelo que decidimos verificar quais os diferentes valores para cada uma das colunas, visto que alguns valores podiam ser agrupados. Por exemplo:
    - caso o identificador de cidade contivesse o nome da cidade este podia ser substituído pelas coordenadas geográficas, em vez de simplesmente ser eliminado. Isto não se verificou pois o identificador era apenas um inteiro.
    - em certos casos existiam colunas do tipo X.Y.Z.W que podiam ser agrupadas
    - ainda noutros casos existiam valores que eram diferentes mas referem o mesmo (por exemplo: On e ON)
Basicamente tentamos tratar os dados, agrupando alguns valores e limpando outros também.
Chegamos à conclusão que a maior parte dos identificadores presentes no *dataset* seriam eliminados pelo facto de serem inteiros e não ser possível agrupá-los, pelo que os que continham muitos valores (que definimos como sendo acima de 100 valores diferentes) foram todos eliminados.
Eliminamos também atributos que apenas continham 1 valor diferente pelo facto de não acrescentarem nada de novo.

Relativamente às colunas que apresentam NAs, é necessário substituí-los. Para isso, verificamos a frequência de cada valor da coluna e caso esta seja superior a 94%, substituímos os NAs pelo valor modal. Caso contrário, prevemos o valor dos NAs.

#### Variáveis quantitativas
O tratamento das variáveis quantitativas foi bastante mais simples, visto que apenas verificamos a existência de *outliers*. Isto é importante pois deve-se substituir os NAs:
    - pela média, se não exisitirem *outliers*;
    - pela mediana, caso existam *outliers*.
E ainda pelo facto de que a *standardização* deve ser realizada com a seguinte equação:
    - (x - média ) / desvio-padrão, caso não existam *outliers*
    - (x - mediana ) / intervalo entre quartis, caso existam
De referir que neste tratamento dos dados para calcular a média, mediana e os restantes valores não usamos os dados de teste e depois aplicamos a equção e substituição a todo o *dataset*.

# FALAR DAS CORRS?

### Variáveis eliminadas por apenas conterem 1 valor distinto

Eliminamos estas variáveis porque uma variável que apenas contém um valor distinto não acrescenta qualquer informação ao conjunto de dados.

In [3]:
ms.nunique()[ms.nunique() == 1]

PuaMode                       1
Census_IsFlightingInternal    1
Census_IsWIMBootEnabled       1
dtype: int64

### Variáveis eliminados por excesso de NAs
Consideramos este número de valores nulos demasiado elevado para realizarmos qualquer presunção acerca dos dados.
Para além disso não temos qualquer noção de algum valor de referência em qualquer variável.

In [4]:
ms[['PuaMode','DefaultBrowsersIdentifier','Census_ProcessorClass']].isna().sum() / ms.shape[0]

PuaMode                      0.999772
DefaultBrowsersIdentifier    0.951490
Census_ProcessorClass        0.995868
dtype: float64

### Variáveis eliminadas por excesso de valores

Eliminamos estas variáveis à partida porque contêm um número demasiado elevado de valores distintos (acima de 1000) onde não é possível agrupar os valores.
Como os métodos de *machine learning* tradicional não conseguem tratabalhar eficientemente com tantos valores de *input* como os de *deep learning* tivemos de limitar um pouco esses valores. Caso mantivessemos todas as variáveis ficavamos com um número de *inputs* na ordem das centenas de milhares.

In [5]:
ms[['CityIdentifier',
'AVProductStatesIdentifier',
'Census_OEMNameIdentifier',
'Census_OEMModelIdentifier',
'Census_ProcessorModelIdentifier',
'Census_FirmwareVersionIdentifier']].nunique()

CityIdentifier                      40323
AVProductStatesIdentifier            6100
Census_OEMNameIdentifier             1664
Census_OEMModelIdentifier           45094
Census_ProcessorModelIdentifier      2331
Census_FirmwareVersionIdentifier    25074
dtype: int64

### Variáveis em risco de serem eliminadas por um excesso de atributos

In [6]:
risco = ['CountryIdentifier','IeVerIdentifier',
    'Census_OSBuildNumber','Census_OSBuildRevision',
    'Census_OSUILocaleIdentifier','GeoNameIdentifier',
   'LocaleEnglishNameIdentifier','Census_FirmwareManufacturerIdentifier',
        'OsBuildLab']
ms[risco].nunique()

CountryIdentifier                        222
IeVerIdentifier                          189
Census_OSBuildNumber                      64
Census_OSBuildRevision                   239
Census_OSUILocaleIdentifier               94
GeoNameIdentifier                        266
LocaleEnglishNameIdentifier              241
Census_FirmwareManufacturerIdentifier    314
OsBuildLab                               472
dtype: int64

In [7]:
for var in risco:
    print(ms[var].value_counts() / ms[var].value_counts().sum() * 100)

43     4.406000
29     3.891667
141    3.707500
93     3.178000
171    3.162333
60     2.604500
201    2.482000
207    2.395500
66     2.315500
89     2.275167
97     2.188167
214    2.156333
158    2.068167
44     2.050667
9      1.955167
107    1.865167
203    1.807500
51     1.797833
41     1.784333
68     1.779500
35     1.555000
160    1.489500
195    1.483333
142    1.470667
149    1.456333
205    1.330333
155    1.237833
164    1.205500
173    1.081000
159    1.027000
         ...   
182    0.008667
74     0.008333
5      0.007833
198    0.007833
215    0.007667
28     0.007333
197    0.007333
123    0.007000
75     0.007000
196    0.006833
192    0.006167
126    0.006167
114    0.006167
105    0.006000
14     0.006000
64     0.005333
79     0.004833
187    0.004500
174    0.004500
200    0.004167
10     0.004000
212    0.003833
216    0.003500
128    0.003167
193    0.002667
217    0.002500
161    0.002500
186    0.002500
165    0.002333
37     0.001667
Name: CountryIdentifier,

17134.1.amd64fre.rs4_release.180410-1804                 40.942803
16299.431.amd64fre.rs3_release_svc_escrow.180502-1908    13.996713
16299.15.amd64fre.rs3_release.170928-1534                10.832369
15063.0.amd64fre.rs2_release.170317-1834                  8.094027
17134.1.x86fre.rs4_release.180410-1804                    2.864343
16299.15.x86fre.rs3_release.170928-1534                   2.614509
14393.2189.amd64fre.rs1_release.180329-1711               2.173007
10240.17443.amd64fre.th1.170602-2340                      1.926840
10586.1176.amd64fre.th2_release_sec.170913-1848           1.659339
15063.0.x86fre.rs2_release.170317-1834                    0.712002
14393.0.amd64fre.rs1_release.160715-1616                  0.651502
9600.19101.amd64fre.winblue_ltsb_escrow.180718-1800       0.628502
9600.19067.amd64fre.winblue_ltsb_escrow.180619-2033       0.617002
16299.637.amd64fre.rs3_release_svc.180808-1748            0.497502
14393.2189.x86fre.rs1_release.180329-1711                 0.43

Decidimos eliminar as variáveis acima descritas, considerando apenas as que aparecem menos de 100 vezes.
Num contexto diferente podíamos ter feito uma análise mais refinada, mas visto que já temos centenas de variáveis decidimos manter apenas as que pareciam relevantes.
Possivelmente o filtro de variância ia eliminar as outras "variáveis" (depois de realizado o *one-hot encoding*) das outras variáveis, mas neste caso optamos por simplificar a análise.

### Variáveis eliminadas por serem semelhantes a outras variáveis

- Census_ChassisTypeName
    - Esta variável possui várias semelhanças com outras duas variáveis já existentes no dataset: "Census_MDC2FormFactor" e "Census_PowerPlatformRoleName". Além disso, esta variável apresenta valores estranhos, tais como "LunchBox" ou "Blade", além de uma série de valores numéricos sem sentido.


In [8]:
ms['Census_ChassisTypeName'].value_counts()

Notebook               352649
Desktop                125724
Laptop                  46314
Portable                24345
AllinOne                13689
MiniTower                5858
Convertible              5714
Other                    5143
UNKNOWN                  4642
Detachable               3448
LowProfileDesktop        3385
HandHeld                 2956
SpaceSaving              2018
Tablet                    896
Tower                     826
Unknown                   687
MainServerChassis         658
MiniPC                    319
LunchBox                  267
RackMountChassis          262
BusExpansionChassis        49
SubNotebook                43
30                         19
0                          10
StickPC                     9
Blade                       5
SealedCasePC                4
PizzaBox                    4
35                          3
MultisystemChassis          3
31                          2
32                          1
28                          1
ExpansionC

###  Eliminação de variáveis.

#### Verificação das variáveis a eliminar

In [9]:
eliminar = ['MachineIdentifier',
 'PuaMode',
 'DefaultBrowsersIdentifier',
 'Census_ProcessorClass',
 'CountryIdentifier',
 'CityIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'AVProductStatesIdentifier',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_FirmwareManufacturerIdentifier',
 'IeVerIdentifier',
 'Census_OSBuildRevision',
 'OsBuildLab',
 'Census_ChassisTypeName',
 'Census_IsFlightingInternal',
 'Census_IsWIMBootEnabled']

In [10]:
ms[eliminar].nunique()

MachineIdentifier                        600000
PuaMode                                       1
DefaultBrowsersIdentifier                   610
Census_ProcessorClass                         3
CountryIdentifier                           222
CityIdentifier                            40323
GeoNameIdentifier                           266
LocaleEnglishNameIdentifier                 241
AVProductStatesIdentifier                  6100
Census_OEMNameIdentifier                   1664
Census_OEMModelIdentifier                 45094
Census_ProcessorModelIdentifier            2331
Census_FirmwareVersionIdentifier          25074
Census_FirmwareManufacturerIdentifier       314
IeVerIdentifier                             189
Census_OSBuildRevision                      239
OsBuildLab                                  472
Census_ChassisTypeName                       35
Census_IsFlightingInternal                    1
Census_IsWIMBootEnabled                       1
dtype: int64

In [11]:
ms[eliminar].isna().sum() / ms.shape[0] * 100

MachineIdentifier                         0.000000
PuaMode                                  99.977167
DefaultBrowsersIdentifier                95.149000
Census_ProcessorClass                    99.586833
CountryIdentifier                         0.000000
CityIdentifier                            3.611333
GeoNameIdentifier                         0.002500
LocaleEnglishNameIdentifier               0.000000
AVProductStatesIdentifier                 0.400500
Census_OEMNameIdentifier                  1.067667
Census_OEMModelIdentifier                 1.147833
Census_ProcessorModelIdentifier           0.482167
Census_FirmwareVersionIdentifier          1.835333
Census_FirmwareManufacturerIdentifier     2.109833
IeVerIdentifier                           0.664167
Census_OSBuildRevision                    0.000000
OsBuildLab                                0.000333
Census_ChassisTypeName                    0.007500
Census_IsFlightingInternal               83.058333
Census_IsWIMBootEnabled        

In [12]:
ms = ms.drop(columns = eliminar, errors='ignore')

### Análise variáveis quantitativas

In [13]:
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']

In [14]:
ms[numericos].describe()

,AVProductsInstalled,AVProductsEnabled,Census_ProcessorCoreCount,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges
count,597597.000000,597597.000000,597107.000000,5.962750e+05,5.962760e+05,594387.000000,596741.000000,596747.000000,596747.000000,5.817450e+05
mean,1.326059,1.020740,3.992517,5.134265e+05,3.778890e+05,6122.585413,16.680267,1548.203957,897.898362,1.124047e+09
std,0.522140,0.167313,2.080830,3.731920e+05,3.247680e+05,5215.878857,5.904098,369.372451,215.199986,1.887927e+09
min,0.000000,0.000000,1.000000,0.000000e+00,0.000000e+00,512.000000,4.900000,-1.000000,-1.000000,0.000000e+00
25%,1.000000,1.000000,2.000000,2.393720e+05,1.208040e+05,4096.000000,13.900000,1366.000000,768.000000,0.000000e+00
50%,1.000000,1.000000,4.000000,4.769400e+05,2.498990e+05,4096.000000,15.500000,1366.000000,768.000000,0.000000e+00
75%,2.000000,1.000000,4.000000,9.538690e+05,4.759780e+05,8192.000000,17.200000,1920.000000,1080.000000,4.294967e+09
max,6.000000,4.000000,88.000000,6.867302e+07,1.144361e+07,917504.000000,142.000000,11520.000000,4320.000000,4.294967e+09


In [15]:
ms[numericos].nunique()

AVProductsInstalled                                       7
AVProductsEnabled                                         5
Census_ProcessorCoreCount                                28
Census_PrimaryDiskTotalCapacity                        1193
Census_SystemVolumeTotalCapacity                     158820
Census_TotalPhysicalRAM                                 626
Census_InternalPrimaryDiagonalDisplaySizeInInches       508
Census_InternalPrimaryDisplayResolutionHorizontal       539
Census_InternalPrimaryDisplayResolutionVertical         568
Census_InternalBatteryNumberOfCharges                  5947
dtype: int64

### Análise variáveis qualitativas

In [16]:
for tipo in ms.columns:
    if tipo not in numericos:
            print(ms[tipo].describe())

count           600000
unique               4
top       win8defender
freq            593588
Name: ProductName, dtype: object
count          600000
unique             53
top       1.1.15200.1
freq           258455
Name: EngineVersion, dtype: object
count              600000
unique                 97
top       4.18.1807.18075
freq               345464
Name: AppVersion, dtype: object
count           600000
unique            6713
top       1.273.1420.0
freq              6926
Name: AvSigVersion, dtype: object
count    600000.000000
mean          0.000010
std           0.003162
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: IsBeta, dtype: float64
count    597831.000000
mean          6.847266
std           1.019600
min           0.000000
25%           7.000000
50%           7.000000
75%           7.000000
max          35.000000
Name: RtpStateBitfield, dtype: float64
count    600000.000000
mean          0.017110
std     

KeyboardInterrupt: 

In [ ]:
categoricos = [i for i in ms.columns if i not in numericos]

In [ ]:
ms[categoricos].isna().sum()[ms[categoricos].isna().sum() != 0] / ms.shape[0] * 100

##### Verificação de variáveis com apenas 1 valor distinto

In [ ]:
for var in ms.columns:
    if len(ms[var].value_counts()) < 2:
        print(var)

In [ ]:
ms[categoricos].nunique()[ms[categoricos].nunique() > 40]

#### Agrupar classes que têm valores do tipo X.Y.Z.W para X.Y

In [ ]:
def trataString(string):
    words = string.split(".")
    palavra= '' + words[0] + '.' + words[1]
    return palavra
ms['AppVersion'] = ms['AppVersion'].apply(lambda x : trataString(x))
print(len(ms['AppVersion'].unique()))
ms['AvSigVersion'] = ms['AvSigVersion'].apply(lambda x : trataString(x))
len(ms['AvSigVersion'].unique())

#### Agrupar classes que têm valores do tipo X.Y.Z.W para X.Y.Z

In [ ]:
def trataString2(string):
    words = string.split(".")
    palavra= '' + words[0] + '.' + words[1] + '.' + words[2]
    return palavra
ms['Census_OSVersion'] = ms['Census_OSVersion'].apply(lambda x : trataString2(x))
print(len(ms['Census_OSVersion'].unique()))

#### Agrupar alguns valores de variáveis

- SmartScreen
    - valores on e On significam o mesmo pelo que podem ser agrupados
    - o mesmo acontece com off, Off e OFF
    - valores que são imperceptíveis são classificados como other
- UacLuaenable
    - valores acima de 1 são considerados como 1 (a variável é binária)

In [ ]:
ms['SmartScreen'] = ms['SmartScreen'].replace({"on": "On", "off": "Off", "OFF": "Off"})
ms['SmartScreen'] = ms['SmartScreen'].replace({"&#x02;": "other", "&#x01;": "other", "0": "other"})
ms['SmartScreen'].value_counts()

In [ ]:
ms.loc[ms['UacLuaenable'] > 1, 'UacLuaenable'] = 1
ms['UacLuaenable'].value_counts()

In [ ]:
ms['Census_InternalBatteryType'].value_counts()

#ms['Census_InternalBatteryType'] = ms['Census_InternalBatteryType'].replace({"li-i": "lion", "&#x01;": "other", "0": "other"})



### Conversão dos valores para categóricos

In [ ]:
for tipo in ms.columns:
    if tipo not in numericos:
            ms[tipo] = ms[tipo].astype('category')    

In [ ]:
ms.describe()

In [ ]:
ms[categoricos].nunique()[ms[categoricos].nunique() > 40]

In [ ]:
ms[categoricos].nunique().sum()

In [ ]:
ms['Census_InternalBatteryType'].nunique()

Apenas podemos substituir diretamente pelo valor modal as variáveis cuja classe modal contenha uma percentagem de valores acima de um *threshold*. 
O *threshold* que nos pareceu mais adequado foi 94%.

In [ ]:
verTratar = []
for j in [i for i in categoricos]:
    if ms[j].isna().sum() > 0:
        a = (ms[j].value_counts() / ms[j].value_counts().sum() * 100)
        if a.iloc[0] < 94:
            verTratar.append(j)

In [ ]:
verTratar

### Tratar NAs dos restantes valores

Ter atenção que existe no mínimo uma coluna que não pode ser tratada com o valor modal

* Census_IsAlwaysOnAlwaysConnectedCapable - 6%
* Wdft_IsGame - 28%

* ainda existem mais, estão no array verTratar

##### Depois ver se existe este problema em algum dos vossos

In [ ]:
tratar = [i for i in categoricos if i not in verTratar]
moda = ms[tratar].mode()
for coluna in tratar:
    if(ms[coluna].isna().sum() > 0):
        ms[coluna].fillna(moda[coluna][0],inplace=True)
print(ms[categoricos].isna().sum().sum())
print(ms[tratar].isna().sum().sum())
print(ms[verTratar].isna().sum().sum())

## Separação do dataset em treino teste

Vamos separar porque o restante tratamento dos NA's e a normalização vão ser baseados em métodos paramétricos pelo que é necessário esta separação. Os parametros serão estimados com os dados de teste e posteriormente aplicados aos dados de teste

In [ ]:
treinoT = 550000
testeT = 600000
treino = ms[:treinoT]
teste = ms[treinoT:testeT]

In [ ]:
print(teste.shape)
print(treino.shape)

### Tratar variáveis 

## Valores numéricos
Tratamos agora os valores numéricos

### Análise outliers

Vamos realizar uma análise aos outliers para perceber se a substituição dos valores quantitativos e posterior *standardização* será realizada com a média ou mediana e desvio-padrão ou intervalo entre quartis.

In [ ]:
print((treino[numericos[0]]).plot.box())


In [ ]:
print((treino[numericos[1]]).plot.box())


In [ ]:
print((treino[numericos[2]]).plot.box())


In [ ]:
print((treino[numericos[3]]).plot.box())


In [ ]:
print((treino[numericos[4]]).plot.box())

In [ ]:
print((treino[numericos[5]]).plot.box())

In [ ]:
print((treino[numericos[6]]).plot.box())

In [ ]:
print((treino[numericos[7]]).plot.box())

In [ ]:
print((treino[numericos[8]]).plot.box())

In [ ]:
print((treino[numericos[9]]).plot.box())

**Variáveis sem outliers**

- Census_InternalBatteryNumberOfCharges
- AVProductsInstalled
- AVProductsEnabled

### Tratar NAs das variáveis numéricas/continuas

Se a variavel tem outliers substituir por mediana. Caso contrário substituir pela média.

In [ ]:
subNum = treino[numericos].median()
subNum['Census_InternalBatteryNumberOfCharges'] = treino['Census_InternalBatteryNumberOfCharges'].mean()
subNum['AVProductsInstalled'] = treino['AVProductsInstalled'].mean()
subNum['AVProductsEnabled'] = treino['AVProductsEnabled'].mean()
q1 = treino[numericos].quantile(0.25)
q3 = treino[numericos].quantile(0.75) 
divisao = q3 - q1
divisao['Census_InternalBatteryNumberOfCharges'] = treino['Census_InternalBatteryNumberOfCharges'].std()
divisao['AVProductsInstalled'] = treino['AVProductsInstalled'].std()
divisao['AVProductsEnabled'] = treino['AVProductsEnabled'].std()
for coluna in numericos:
    if(ms[coluna].isna().sum() > 0):
        ms[coluna].fillna(subNum[coluna],inplace=True)
ms[numericos].isna().sum().sum()

In [ ]:
ms.isna().sum().sum()

## Standardização dos dados numéricos

In [ ]:
divisao

In [ ]:
treinoStd = ( treino[numericos] - subNum ) / divisao
treinoStd.describe()

### Realizar one-hot encoding
Decidimos codificar uma função que realiza o one-hot-encoding das variáveis necessárias

In [ ]:
import gc
def oneHotEncoding(serie):
    colunas = serie.unique()
    dic = {}
    for coluna in colunas:
        dic[serie.name + '_' + str(coluna)] = []
    for i in serie:
        dic[serie.name + '_' + str(i)].append(1)
        for j in colunas:
            if j != i:
                dic[serie.name + '_' + str(j)].append(0)
    r = pd.DataFrame(dic)
    del dic
    gc.collect()
    return r

Para cada variável que podia ser tratada e fosse necessário o one-hot encoding realizamos o mesmo e gravamos num ficheiro.
<code>encode = [i for i in tratar if len(ms[i].unique()) > 2]
for var in encode:
        aux = oneHotEncoding(ms[var])
        aux.to_csv('./encoding/' + var +'-1hot.csv', sep=',', encoding='utf-8')</code>
De seguida juntamos todas as variáveis que já tivessem sido tratadfas num único ficheiro.

### Prever os restantes com Random Forests
De seguida vamos prever os valores dos NAs das variáveis que faltam ser tratadas. 
Para isso apenas realizamos uma Random Forest e prevemos o valor, sendo que a random forest apenas é treinada com os dados de treino considerados (não pode usar os de teste).

Temos plena noção que o ideal seria tentar arranjar o melhor modelo para prever estes NAs fazendo vários modelos e vendo qual seria o melhor (basicamente o mesmo que vamos realizar para prever se tem deteções), no entanto devido à falta de tempo não nos foi possível utilizar esse método.

## Definir os tipos
Tivemos de definir os tipos das variáveis categóricas para ser possível carregar o conjunto de dados

In [ ]:
import pandas as pd
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']
dtype = {}
for df in pd.read_csv('encoding/final_sembat.csv',low_memory=False,chunksize=10):
    for var in df.columns:
        if var not in numericos:
            dtype[var] = 'int8'
    break

Tivemos de definir os tipos para int8 para que não ocupe demasiado espaço, caso contrário não era possível carregar o dataset. Para além disso tivemos de ir lendo o dataset em *chunks* e juntando-os pois a função **read_csv** tem problemas de memória.

In [ ]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('encoding/final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
#     if i == 55:
#         break
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)
#aux = pd.read_csv('encoding/predict.csv',low_memory=False,dtype=dtype)

Função que realiza o treino do modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def realizaSVM(inp,out):
    is_na = out.isna()
    indice = [i for i in range(len(is_na)) if is_na[i] == False]
    auxI = inp.iloc[indice]
    auxO = out.iloc[indice]
    neigh = RandomForestClassifier(n_estimators=10, max_depth=50)
    print(auxO.shape)
    neigh.fit(auxI,auxO)
    return neigh

Função que preenche os valoers nulos consoante o modelo retornado pela função anterior. De seguida realiza o one-hot encoding e grava num ficheiro.

In [ ]:
def realizaPrevEOfC(nome,treinoT,aux,ms):
    oi = ms[nome]
    oi = oi.astype('category')
    svm_oi = realizaSVM(aux[:treinoT],oi[:treinoT])
    is_na = oi.isna()
    indice = [i for i in range(len(is_na)) if is_na[i] == True]
    X_Prev = aux.iloc[indice]
    previsto = svm_oi.predict(X_Prev)
    for i in range(len(indice)):
        oi[indice[i]] = previsto[i]
    print(oi.isna().sum())
    if oi.nunique() > 2:
        gr = oneHotEncoding(oi)
    else:
        gr = oi
    gr.to_csv('./encoding/' + oi.name +'-1hot.csv', sep=',', encoding='utf-8')

Percorremos todas as variáveis que faltavam tratar e aplicamos o método descrito anteriormente.

<code>for var in verTratar:
    treinoT = 550000
    realizaPrevEOfC('Wdft_RegionIdentifier',treinoT,auxPred,ms)</code>

### Realizar variance threshold
### Realizar os modelos
    - n se esqueçam dos hiper-parametros
    - n se esqueçam de usar tanto o conjuntos de dados todo como do conjunto dado pelo filtro de variância

In [ ]:
trainX = auxPred.loc[:499999, auxPred.columns != 'HasDetections']
valX = auxPred.loc[500000:549999, auxPred.columns != 'HasDetections']
testX = auxPred.loc[550000:599999, auxPred.columns != 'HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']
valY = auxPred.loc[550000:599999,'HasDetections']
print(trainX.shape)
print(valX.shape)
print(trainY.shape)
print(valY.shape)

## Variance threshold
Para redução da dimensionalidade realizamos um filtro de variância. Obviamente podíamos ter usado outros métodos como **SelectKBest** ou **SelectPercentile**, no entanto decidimos usar apenas o filtro de variância pelo facto de os modelos lidarem bem com muitas colunas e porque não tínhamos muito tempo para perceber quantos preditores devíamos considerar (podia ser um "hiper-parâmetro"). Inicialmente a ideia até passou por usar **PCA**, no entanto a mesma foi logo desconsiderada pelo facto de existirem demasiadas variáveis categóricas.

Não quisemos definir um filtro de variância demasiado alto, pois possibilitava a eliminação de demasiadas colunas e definimos o limite em 0.1%.

In [ ]:
# import gc
# import math
# from sklearn.feature_selection import VarianceThreshold
# def realizaVarThreshold():
#     indices = []
#     col = trainX.columns
#     total = len(col)
#     chunk = math.floor(total / 10)
#     print(chunk)
#     quantos = 0
#     for i in range(chunk):
#         sel = VarianceThreshold(threshold=0.001)
#         try:
#             sel.fit(trainX[col[quantos:quantos+10]])
#             aux = [i+quantos for i in sel.get_support(indices=True)]
#             indices.extend(aux)
#         except:
#             pass
#         del sel
#         gc.collect()
#         quantos = quantos+10
#     sel = VarianceThreshold(threshold=0.001)
#     try:
#         sel.fit(trainX[col[quantos:quantos+7]])
#         indices.extend(sel.get_support(indices=True))
#     except:
#         pass
#     del sel
#     gc.collect()
#     return indices

In [ ]:
# indices = realizaVarThreshold()
# print(len(indices))
# col = []
# coln = trainX.columns
# for i in indices:
#     col.append(coln[i])

In [ ]:
# trainX[col]

## Seleção de modelos

### Otimização de hiper-parâmetros

Realizamos uma função que realiza uma pesquisa aleatório para otimização de hyper-parâmetros.

Basicamente a função recebe um conjunto (dicionário) dos parâmetros possíveis e seleciona 10 combinações aleatórias retiradas desse conjunto. Depois para cada uma das combinações realiza o fit do modelo (que é passado como parâmetro) e faz o *score* do mesmo consoante os dados de validação. Todos os dados são pasados como parâmetro.

Decidimos usar para o *score* o método **AUC** pelo facto de ser este o utilizado na competição na qual este conjunto de dados foi lançado.

In [ ]:
from sklearn.model_selection import ParameterSampler
import copy
import gc
from sklearn.metrics import roc_curve, auc
def hyper_parameter_optimization(params, modeloG, testX, testY, trainX, trainY):
    lista_parametros = list(ParameterSampler(params, n_iter=10, random_state=10))
    for elem in lista_parametros:
        altera = False
        for key, value in elem.items():
            if key == 'solver':
                if value in ['newton-cg','lbfgs','sag']:
                    altera = True
        if altera == True:
            elem['penalty'] = 'l2'
    tabelaFinal = []
    for param in lista_parametros:
        print(param)
        modelo = copy.copy(modeloG)
        modelo.set_params(**param)
        #print(modelo.get_params())
        modelo.fit(trainX,trainY)
        pred = modelo.predict(testX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(testY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        elem = param.copy()
        elem['score'] = score
        tabelaFinal.append(elem)
        del modelo
        gc.collect()
    return tabelaFinal

In [ ]:
raise

### Regressão Logística

Devido às limitações de poder computacional do grupo, apenas foi possível realizar o treino dos modelos com 100000 exemplos. Deste modo selecionamos 100000 exemplos dos 500000 definidos para o treino e treinamos os nossos modelos de regressão logística com esses exemplos.

In [ ]:
trainX2 = trainX[:99999]
trainY2 = trainY[:99999]

Com o objetivo de obter vários modelos com parâmetros diferentes, utilizamos a função *hyper_parameter_optimization* descrita acima. Como parâmetros de estimação, decidimos tentar vários valores para o inverso do parâmetro de regularização, **C**, para a norma utilizada na penalização, **penalty**, e para o algoritmo de otimização, **solver**. No excerto de código seguinte é possível verificar a gama de valores na qual variamos cada um destes três parâmetros.

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

parametros = {'C':[0.001,0.01,0.1,1],
              'penalty':['l1','l2'],
              'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

logisticRegr = LogisticRegression(max_iter=100)

modelosRegLog = hyper_parameter_optimization(parametros,logisticRegr,valX,valY,trainX2,trainY2)

#### Tabela com os parâmetros dos 10 modelos testados
De seguida é possível observar scores de 10 modelos originados por 10 conjuntos de parâmetros. De notar que estes 10 conjuntos são escolhidos aleatoriamente.

In [ ]:
tabelaRLog = pd.DataFrame(modelosRegLog)
tabelaRLog.sort_values(by='score',ascending=False)

De modo a não ser necessário efetuar recorrentemente os mesmos cálculos (muito dispendiosos em termos de tempo de execução), fizemos uma análise mais aprufundada, com 25 combinações diferentes dos parâmetros apresentados acima e armazenamos os resultados dos melhores modelos. Na seguinte tabela podemos observar os 6 modelos que obtiveram melhor pontuação:

In [23]:
import pandas as pd
d = {'C': [1.000, 0.010, 0.001, 0.100, 0.010, 1.000], 
     'penalty': ['l1', 'l1', 'l2', 'l2', 'l2', 'l2'],
     'solver':['liblinear', 'liblinear', 'newton-cg', 'newton-cg', 'newton-cg', 'newton-cg'],
     'score': [0.65226, 0.64746, 0.62702, 0.61828, 0.60736, 0.54390]}
df = pd.DataFrame(data=d)
df

,C,penalty,solver,score
0,1.000,l1,liblinear,0.65226
1,0.010,l1,liblinear,0.64746
2,0.001,l2,newton-cg,0.62702
3,0.100,l2,newton-cg,0.61828
4,0.010,l2,newton-cg,0.60736
5,1.000,l2,newton-cg,0.54390


### Regressão Logística: Variance threshold = 0.001

Seguem-se os resultados obtidos com os dados de treino após se aplicar um threshold de variância nos mesmos.

In [ ]:
parametros = {'C':[0.001,0.01,0.1,1],
              'penalty':['l1','l2'],
              'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

logisticRegr = LogisticRegression(max_iter=100)

modelosRegLog2 = hyper_parameter_optimization(parametros,logisticRegr,valX[col],valY,trainX2[col],trainY2)

#### Tabela com os parâmetros dos 10 modelos testados

In [ ]:
tabelaRLog2 = pd.DataFrame(modelosRegLog2)
tabelaRLog2.sort_values(by='score',ascending=False)

À semelhança do teste anterior, fizemos uma análise com 25 combinações diferentes dos parâmetros e registamos o valor dos melhores modelos. Na seguinte tabela podemos observar os 6 modelos que obtiveram melhor pontuação:

In [24]:
import pandas as pd
d = {'C': [1.000, 0.010, 1.000, 0.010, 0.100, 0.001], 
     'penalty': ['l1', 'l1', 'l2', 'l2', 'l2', 'l2'],
     'solver':['liblinear', 'liblinear', 'newton-cg', 'newton-cg', 'newton-cg', 'newton-cg'],
     'score': [0.65228, 0.64738, 0.62150, 0.61460, 0.61294, 0.54404]}
df = pd.DataFrame(data=d)
df

,C,penalty,solver,score
0,1.000,l1,liblinear,0.65228
1,0.010,l1,liblinear,0.64738
2,1.000,l2,newton-cg,0.62150
3,0.010,l2,newton-cg,0.61460
4,0.100,l2,newton-cg,0.61294
5,0.001,l2,newton-cg,0.54404


## Random forests

Relativamente aos hiper-parâmetros que foram considerados para o método de **Random Forests** os mesmos foram os seguintes:
    - *n_estimators* - o número de árvores usadas para prever
    - *max_features* - o máximo número de preditores considerado (apenas consideramos 2 valores, que correspondem à raiz quadrada ou ao logaritmo)
    - *max_depth* - a profundidade máxima da árvore
    - *criterion* - o critério de seleção na decisão

Estes foram os parâmteros que nos pareceram mais interessantes, talvez por ordem de insteresse. Não conseguimos utilizar muito bem o que queríamos, porque por exemplo gostavamos de ter utilizado números mais elevados de estimadores ou não definindo limites ao número de variáveis. Não foi possível pelo facto de o conjunto de dados ser demasiado grande o que não permitia que o modelo conseguisse realizar o *fit*.
De referir apenas que com os dados obtidos com o filtro de variância conseguimos utilizar 100 estimadores pois eram menos dados.

#### Com todos os atributos

In [19]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [4, 8, 16, 32, 64]#, 100, 200]
max_features = ['auto','log2']
max_depth = [32,64,128,None]
criterion = ['gini','entropy']
parametros = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'criterion':criterion}
m = RandomForestClassifier()
tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

In [20]:
import pandas as pd
tab = pd.DataFrame(tabela)
tab.sort_values(by='score',ascending=False)

,criterion,max_depth,max_features,n_estimators,score
7,gini,32.0,auto,32,0.661923
9,gini,64.0,log2,64,0.655149
0,entropy,32.0,auto,8,0.646573
3,gini,NaN,auto,16,0.644740
8,gini,NaN,log2,16,0.636702
1,entropy,32.0,log2,8,0.633819
6,gini,128.0,auto,8,0.628578
2,gini,128.0,log2,8,0.615860
5,entropy,128.0,log2,4,0.601088
4,gini,NaN,log2,4,0.596266


#### Apenas com os atributos que passaram o filtro de variância

In [ ]:
trainX = auxPred.loc[:499999, col]
valX = auxPred.loc[500000:549999, col]
from sklearn.ensemble import RandomForestClassifier
n_estimators = [8, 16, 32, 64, 100]
max_features = ['auto','log2']
max_depth = [32,64,128,None]
criterion = ['gini','entropy']
parametros = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'criterion':criterion}
m = RandomForestClassifier()
tabelaVT = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

In [22]:
import pandas as pd
tabVT = pd.DataFrame(tabelaVT)
tabVT.sort_values(by='score',ascending=False)

,criterion,max_depth,max_features,n_estimators,score
7,gini,32.0,auto,64,0.667463
9,gini,64.0,log2,100,0.658444
0,entropy,32.0,auto,16,0.657307
3,gini,NaN,auto,32,0.655033
1,entropy,32.0,log2,16,0.652853
8,gini,NaN,log2,32,0.649633
6,gini,128.0,auto,16,0.643742
2,gini,128.0,log2,16,0.640022
5,entropy,128.0,log2,8,0.629723
4,gini,NaN,log2,8,0.626342
